Split AVL Route points into N and S points<br>

Split GTFS routes into N and S routes<br>
Create Routes layer for the routes*<br>

Create a route from line features

In [1]:
import arcpy
arcpy.CreateFileGDB_management(r"C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Linear_Ref_Files", "LRF.gdb")

<Result 'C:\\Users\\Cole\\Desktop\\Spring2022\\AVLResearch\\Data\\Linear_Ref_Files\\LRF.gdb'>

In [2]:
import pandas as pd
AVLs = r'C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Aline-vehicle-messages-Oct1_Oct8_2016\Aline-vehicle-messages-Oct1_Oct8-2016.csv'
GTFS = r':\Users\Cole\Desktop\Spring2022\AVLResearch\Data\GTFS_Query'

AVL_Points = pd.read_csv(AVLs, index_col = False)

In [3]:
AVL_Points_Sort = AVL_Points.sort_values(by =['SOURCE_HOST','MESSAGE_TIMESTAMP','LOCAL_TIMESTAMP'])

In [4]:
new = AVL_Points_Sort[["SOURCE_HOST", 'LONGITUDE',"LATITUDE",
                       'LOCAL_TIMESTAMP', 'MESSAGE_TIMESTAMP',
                       'MESSAGE_TYPE_TEXT', 'DIRECTION',
                      'ODOMETER', 'FOM']]
AVL_Points_Select = new[101:209]

In [5]:
AVL_Points_Select["LongCoord"] = AVL_Points_Select['LONGITUDE']/10000000
AVL_Points_Select["LatCoord"] = AVL_Points_Select['LATITUDE']/10000000

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
AVL_Points_Select.to_csv(r"C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Linear_Ref_Files\AVL_Pts.csv")

In [7]:
arcpy.env.workspace = r"C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Linear_Ref_Files"
arcpy.management.XYTableToPoint(r"AVL_Pts.csv", "LRF.gdb\AVL_Pts_shp",
                                "LongCoord", "LatCoord", "")
    
    
input_features = "LRF.gdb\AVL_Pts_shp"

# output data
output_feature_class = r"LRF.gdb\AVL_Pts_Proj"

# create a spatial reference object for the output coordinate system
out_coordinate_system = arcpy.SpatialReference('NAD 1983 NSRS2007 StatePlane Minnesota Central FIPS 2202 (Meters)')

# run the tool
arcpy.Project_management(input_features, output_feature_class, out_coordinate_system)

<Result 'C:\\Users\\Cole\\Desktop\\Spring2022\\AVLResearch\\Data\\Linear_Ref_Files\\LRF.gdb\\AVL_Pts_Proj'>

In [8]:
#GTFS shapes to features
#reproject

arcpy.conversion.GTFSShapesToFeatures(r"C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\GTFS_Query\shapes.txt",
                                      r"C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Linear_Ref_Files\LRF.gdb\GTFSShapes")

<Result 'C:\\Users\\Cole\\Desktop\\Spring2022\\AVLResearch\\Data\\Linear_Ref_Files\\LRF.gdb\\GTFSShapes'>

In [9]:
input_features = "LRF.gdb\GTFSShapes"

# output data
output_feature_class = r"LRF.gdb\GTFS_Shapes_Proj"

# create a spatial reference object for the output coordinate system
out_coordinate_system = arcpy.SpatialReference('NAD 1983 NSRS2007 StatePlane Minnesota Central FIPS 2202 (Meters)')

# run the tool
arcpy.Project_management(input_features, output_feature_class, out_coordinate_system)

<Result 'C:\\Users\\Cole\\Desktop\\Spring2022\\AVLResearch\\Data\\Linear_Ref_Files\\LRF.gdb\\GTFS_Shapes_Proj'>

In [11]:
# Set the workspace
arcpy.env.workspace = r'C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Linear_Ref_Files\LRF.gdb'

query = "\"shape_id\" = \'9210001\'"
# Within selected features, further select only those cities with a 
# population > 10,000   
select = arcpy.SelectLayerByAttribute_management('GTFS_Shapes_Proj', 'NEW_SELECTION', 
                                        query)

# Write the selected features to a new feature class
arcpy.CopyFeatures_management(select, 'GTFS_Shapes_Proj_N')

query = "\"shape_id\" = \'9210002\'"
# Within selected features, further select only those cities with a 
# population > 10,000   
select = arcpy.SelectLayerByAttribute_management('GTFS_Shapes_Proj', 'NEW_SELECTION', 
                                        query)

# Write the selected features to a new feature class
arcpy.CopyFeatures_management(select, 'GTFS_Shapes_Proj_S')

<Result 'C:\\Users\\Cole\\Desktop\\Spring2022\\AVLResearch\\Data\\Linear_Ref_Files\\LRF.gdb\\GTFS_Shapes_Proj_S'>

In [12]:
# Set the workspace
arcpy.env.workspace = r'C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Linear_Ref_Files\LRF.gdb'
query = "\"DIRECTION\" = 4"   
select = arcpy.SelectLayerByAttribute_management('AVL_Pts_Proj', 'NEW_SELECTION', 
                                        query)

# Write the selected features to a new feature class
arcpy.CopyFeatures_management(select, 'AVL_Pts_Proj_N')

query = "\"DIRECTION\" = 1"
# Within selected features, further select only those cities with a 
# population > 10,000   
select = arcpy.SelectLayerByAttribute_management('AVL_Pts_Proj', 'NEW_SELECTION', query)

# Write the selected features to a new feature class
arcpy.CopyFeatures_management(select, 'AVL_Pts_Proj_S')

<Result 'C:\\Users\\Cole\\Desktop\\Spring2022\\AVLResearch\\Data\\Linear_Ref_Files\\LRF.gdb\\AVL_Pts_Proj_S'>

In [20]:
in_lines = r"C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Linear_Ref_Files\LRF.gdb\GTFS_Shapes_Proj_N"
rid = 'shape_id'
out_routes = r"C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Linear_Ref_Files\LRF.gdb\RouteN"


arcpy.CreateRoutes_lr(in_lines, rid, out_routes, "LENGTH", "#", "#", 
                      "LOWER_LEFT", 0.00018939394)

<Result 'C:\\Users\\Cole\\Desktop\\Spring2022\\AVLResearch\\Data\\Linear_Ref_Files\\LRF.gdb\\RouteN'>

In [21]:
in_lines = r"C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Linear_Ref_Files\LRF.gdb\GTFS_Shapes_Proj_S"
rid = 'shape_id'
out_routes = r"C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Linear_Ref_Files\LRF.gdb\RouteS"


arcpy.CreateRoutes_lr(in_lines, rid, out_routes, "LENGTH", "#", "#", 
                      "UPPER_RIGHT", 0.00018939394)

<Result 'C:\\Users\\Cole\\Desktop\\Spring2022\\AVLResearch\\Data\\Linear_Ref_Files\\LRF.gdb\\RouteS'>

In [6]:
import arcpy
#locate features along route
arcpy.env.workspace = r'C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Linear_Ref_Files\LRF.gdb'


# Set local variables
feats = "AVL_Pts_Proj_S"
rts = "RouteS"
rid = "shape_id"
tol = "9 Meters"
tbl = "locate_events_S_T4"
props = "shape_id POINT mp" 

# Execute LocateFeaturesAlongRoutes
arcpy.LocateFeaturesAlongRoutes_lr(feats,
                                   rts,
                                   rid,
                                   tol,
                                   tbl,
                                   props,
                                   route_locations = True,
                                   distance_field = True,
                                   m_direction_offsetting = True)

id,value
0,C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Linear_Ref_Files\LRF.gdb\locate_events_S_T4
1,shape_id POINT mp


In [ ]:
# hm, when placing points from locate table 1984 works bu not the NAD StatePlane